# Method 1: SVM

## Read data

In [ ]:
!pip install rarfile

from google.colab import auth
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials
from pydrive.drive import GoogleDrive
from rarfile import RarFile
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1OS8iDV2TjI4nnixNV7KjQypUEAv9n714'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('train.rar')
with RarFile('train.rar', 'r') as zip:
    zip.extractall(path='./data')
train_data = pd.read_csv('data/train.csv')
train_data

,product_id,product_title,user_id,likes,dislikes,title,comment,advantages,disadvantages,recommend
0,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,3862150,4,1,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]",recommended
1,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,626843,1,0,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN,not_recommended
2,133722,لپ تاپ 15 اینچی ایسوس مدل N501VW - A,497032,6,1,اقرار بیش از حد در ایراد گرفتن,سلام دوستان،،_x000D_\nمنم مثه بعضی از دوستان ق...,"[""همه چیز که تو مشخصات اومده بجز 3 مورد کوچیک""]","[""تاچ پدش میتونست بهتر باشه از نظر سرعت پاسخ گ...",recommended
3,148509,اسپیکر بلوتوثی دیووم مدل Bluetune bean,574130,3,0,بسیار خوب,من چند سالی هست که این اسپیکرو خریدم و واقعا ح...,"[""کیفیت ساخت بسیار خوب\r"",""کیفیت صدا بسیار عال...","[""ندارد""]",recommended
4,5107,هارددیسک اکسترنال سیگیت مدل Backup Plus Deskto...,794316,1,0,هارد با کیفیت و ظاهر شیک,نزدیک به دو ساله که این هارد رو خریدم. استفاده...,"[""کیفیت ساخت خوب\r"",""ظاهر ساده و زیبا\r"",""صدای...","[""عدم مقاومت در برابر ضربه\r"",""عدم سهولت جا به...",recommended
...,...,...,...,...,...,...,...,...,...,...
48077,842976,لیوان گامین گلس مدل محک بسته 6 عددی,2333505,10,1,بی کیفیت,کاملابی کیفیت، تو هر قیمتی اصلا ارزش خرید نداره,NaN,NaN,not_recommended
48078,362029,کتاب سفید و قرمز نسخه بارسلونا,8369747,0,0,عالی بود خون دن داره,عالیییییییییییییییییییی,NaN,NaN,recommended
48079,818381,بالش بادی پورگونر مدل KMG,8027276,0,0,ارزش خرید نداره,اندازه بالش بسیار کوچک هست بیشتر بدرد بچه ۲ سا...,NaN,NaN,not_recommended
48080,549845,شید آباژور آرام مدل SH45/01,4979401,0,0,بسیار تمیز و زیبا,بسیار تمیز و زیبا...من که خیلی راضیم,"[""کاملا مطابق با عکس\r"",""ساخت بسیار تمیز""]","[""ندارد""]",recommended


In [ ]:
# train_data = train_data.drop(['product_id'], axis=1)
# train_data = train_data.drop(['product_title'], axis=1)
# train_data = train_data.drop(['user_id'], axis=1)
# train_data = train_data.drop(['likes'], axis=1)
# train_data = train_data.drop(['dislikes'], axis=1)
# train_data = train_data.drop(['advantages'], axis=1)
# train_data = train_data.drop(['disadvantages'], axis=1)

In [ ]:
# for column_name in train_data.columns:
#         train_data[column_name] = train_data[column_name].fillna(train_data[column_name].mode()[0])

### Split train and target columns

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_target = le.fit_transform(train_data.recommend)
train_data = train_data.title + " " + train_data.comment
train_data = train_data.astype(str)
# train_data.comment = train_data.comment.astype(str)
# train_data.title = train_data.title.astype(str)
# train_data = train_data.drop(['recommend'], axis=1)

### Split train and validation data

In [ ]:
from sklearn.model_selection import train_test_split

train_x, validation_x, train_y, validation_y = train_test_split(train_data, train_target, test_size = 0.1, random_state=42)
# train_x, train_y = train_data, train_target

## Preporocess data

In [ ]:
!pip install hazm

from hazm import *
import re
import string


normalizer = Normalizer(persian_numbers=False)
lemmatizer = Lemmatizer()
stop_words = stopwords_list()

def remove_hyperlink(word):
    word = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", "", word[::-1])
    word = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", "", word[::-1])
    return word

def to_lower(word):
    return word.lower()

def remove_number(word):
    return re.sub(r'\d+', '', word)

def remove_punctuation(word):
    word = word.translate(str.maketrans(dict.fromkeys(string.punctuation)))
    return word.replace("،", "")
    
def remove_whitespace(word):
    return word.strip()

def replace_newline(word):
    return word.replace('\n','')

def remove_stop_words_and_lammatize(word):
    tokens = word_tokenize(word)
    new_tokens = [lemmatizer.lemmatize(x) for x in tokens]
    new_word = ' '.join(new_tokens)
    return new_word

def normalize(word):
    return normalizer.normalize(word)

def removeHtmlTags(word):
    return re.sub('<[^>]*>', ' ', word)

def removeEnglishLetters(word):
    return re.sub('[A-Za-z]',' ', word)

def removeUnnecessaryPersianCharacters(word):
    return re.sub('[ء]',' ', word)

def clean_up_pipeline(sentence):
    cleaning_utils = [
                    #   to_lower,
                    #   remove_hyperlink,
                    #   replace_newline,
                    #   remove_number,
                    #   remove_punctuation,
                    #   remove_whitespace,
                      normalize,
                    #   remove_stop_words_and_lammatize,
                    #   removeHtmlTags,
                    #   removeEnglishLetters,
                    #   removeUnnecessaryPersianCharacters
                     ]
    for o in cleaning_utils:
        sentence = o(sentence)
    return sentence

In [ ]:
# print(train_x[1])
train_x = [clean_up_pipeline(o) for o in train_x]
# print(train_x[1])

## Train model

In [ ]:
from sklearn import svm
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC

text_clf_svc = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('tfidf', TfidfTransformer()),
                     ('clf-svm', LinearSVC(C=0.7, multi_class="crammer_singer", max_iter=50000)),
])

text_clf_svc = text_clf_svc.fit(train_x, train_y)
predicted = text_clf_svc.predict(validation_x)
np.mean(predicted == validation_y)

0.9378249116240382

## Predict test dataset

In [ ]:
from google.colab import files
import io

uploaded = files.upload()
test = pd.read_csv(io.BytesIO(uploaded['CI_test.csv']))
test_x = test.title + " " + test.comment
test_x = test_x.astype(str)

In [ ]:
x_test = [clean_up_pipeline(o) for o in test_x]
predicted_svc = text_clf_svc.predict(x_test)
pred_test = le.inverse_transform(predicted_svc)
pred_test

array(['recommended', 'not_recommended', 'not_recommended', ...,
       'recommended', 'not_recommended', 'not_recommended'], dtype=object)

In [ ]:
output = pd.DataFrame({'id': test.id,
                       'recommend': pred_test})
output.to_csv('submission4.csv', index=False)

# Method 2: RNN

## Read data

In [ ]:
raw_data = pd.read_csv('data/train.csv')

## Preprocess data

In [ ]:
def clean_data(raw_data):
    structured_data = []
    for _, row in raw_data.iterrows():
        main_text = str(row['title']) + ' ' + str(row['comment'])
        likes = row['likes']
        dislikes = row['dislikes']
        recommend = row['recommend']

        tokens = tokenizer(main_text)
        structured_data.append({
            'recommend': 1 if recommend == 'recommended' else 0,
            'tokens': tokens
        })
    return structured_data

In [ ]:
def tokenizer(text):
    text = clean_up_pipeline(text)
    text = re.sub('\s+', ' ', text).strip()
    return word_tokenize(text)

In [ ]:
def hash_words(comments):
    temp = [token for token in (comment['tokens'] for comment in comments)];
    res = set()
    for t in temp:
      for b in t:
        res.add(b)
    return res

In [ ]:
def index_words(hashed_word):
    result = {w: index for index, w in enumerate(hashed_word)}
    result['reserved_for_new_words'] = len(hashed_word) + 1
    return result

In [ ]:
def compute_train_data(structured_data, indexed_words):
    x, y = [], []
    for data in structured_data:
        x.append([indexed_words[token] for token in data['tokens']])
        y.append(data['recommend'])
    return np.asarray(x), np.asarray(y)

## Define model

In [ ]:
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.layers.wrappers import Bidirectional
from keras.models import Sequential

def model_generator(word_number):
    model = Sequential()
    
    model.add(Embedding(word_number + 1, 110))
    model.add(Bidirectional(LSTM(110, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    model.summary()
    return model

## Train model

In [ ]:
from keras.preprocessing import sequence

structured_data = clean_data(raw_data)
hashed_word = hash_words(structured_data)
indexed_words = index_words(hashed_word)
x, y = compute_train_data(structured_data, indexed_words)

x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=0.1, random_state=42)

max_length_of_comment = 200
x_train = sequence.pad_sequences(x_train, maxlen=max_length_of_comment)
x_validation = sequence.pad_sequences(x_validation, maxlen=max_length_of_comment)

In [ ]:
batch_size = 20
epoch = 1
model = model_generator(len(indexed_words))
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epoch,
              validation_data=(x_validation, y_validation))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 110)         5828020   
                                                                 
 bidirectional_8 (Bidirectio  (None, 220)              194480    
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 2)                 442       
                                                                 
Total params: 6,022,942
Trainable params: 6,022,942
Non-trainable params: 0
_________________________________________________________________
2164/2164 [==============================] - 2014s 926ms/step - loss: 0.2280 - accuracy: 0.9144 - val_loss: 0.1686 - val_accuracy: 0.9372


## Predict test data

In [ ]:
raw_data_test = pd.read_csv(io.BytesIO(uploaded['CI_test.csv']))

predicts_test = []
for index, row in raw_data_test.iterrows():
    pred_dict = dict()

    title = row['title']
    comment = row['comment']
    text = str(title) + ' ' + str(comment)

    
    tokens = tokenizer(text)
    indexes_of_tokens = [[indexed_words.get(token, indexed_words['reserved_for_new_words']) for token in tokens]]

    seq_result = sequence.pad_sequences(indexes_of_tokens, maxlen=max_length_of_comment)

    result = model.predict(seq_result)
    pred_dict['id'] = row['id']
    if result[0][0] > result[0][1]:
      pred_dict['recommend'] = 'not_recommended'
    else:
      pred_dict['recommend'] = 'recommended'

    predicts_test.append(pred_dict)

In [ ]:
import csv

keys = predicts_test[0].keys()

with open('submission10.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(predicts_test)